In [4]:
from datetime import datetime

class Forum:

    def init(self):
        self.messages = []

    def recupDate(self):
        dateBrute = str(datetime.now())
        date = dateBrute[11:13] + "h" + dateBrute[14:16] + "m" + dateBrute[17:19] + "s"
        return date

    def calculDate(self, date) :
            msg_date = date.split()
            date = ""
            i=0
            while msg_date[i].isdigit() :
                date += msg_date[i]
                i+=1
            heure = int(date)
            while msg_date[i].isdigit() :
                date += msg_date[i]
                i+=1
            minute = int(date)
            while len(msg_date) and msg_date[i].isdigit() :
                date += msg_date[i]
                i+=1
            seconde = int(date)
            total = 3600*heure+60*minute+seconde
            return (total)
    
    def recupererMessagesAnonymes(self, date_debut, date_fin) :
        for msg in self.messages :
            date = self.calculDate(msg[1])
            date_start = self.calculDate(date_debut)
            date_end = self.calculDate(date_fin)
            if date > date_start and date < date_end :
                print(msg)

    def posterMessageAnonyme(self, message) :
        date = self.recupDate()
        self.messages.append([message, date])



In [5]:
if __name__ == "__main__" :
    print("start")
    forum = Forum()
    while(True) :
        string = input()
        if "show_message" == string :
            print("entrez la date au format 11h02m03s" )
            date_debut = input()
            date_fin = input()
            Forum.recupererMessagesAnonymes()
        else :
            Forum.posterMessageAnonyme(string)

# <b> Canal de diffusion anonyme et génération de secret </b>

<i> Compte-rendu du TP2 par Valentin SÉRÉE et Jérémy SERENA [INFO3]</i>